# Imports

In [86]:
import os
import io
import sys
import ollama
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

# Environment

In [87]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

# Initialize

In [88]:
openai = OpenAI()

MODEL_GPT_4O = "gpt-4o"
MODEL_GPT_4_TURBO = "gpt-4-turbo"
MODEL_GPT_35_TURBO = "gpt-3.5-turbo"
MODEL_LLAMA = 'llama3.2'

# Prompt

In [89]:
def system_prompt_for(language):
    system_prompt = f"You are an assistant that reimplements {language} code in high performance C++. "
    system_prompt += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
    system_prompt += "The C++ response needs to produce an identical output in the fastest possible time."
    return system_prompt

In [91]:
def user_prompt_for(language):
    user_prompt = f"Rewrite this {language} code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += language
    return user_prompt

In [92]:
def messages_for(language):
    return [
        {"role": "system", "content": system_prompt_for(language)},
        {"role": "user", "content": user_prompt_for(language)}
    ]

# Write to a file called optimized.cpp

In [93]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [94]:
def stream_gpt(language, OPENAI_MODEL):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(language), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [95]:
def optimize(language, model):
    if model==MODEL_GPT_4O or model==MODEL_GPT_4_TURBO or model==MODEL_GPT_35_TURBO:
        result = stream_gpt(language, model)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far 

# Execute

In [96]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [97]:
def execute_cobol(code):
    import subprocess, tempfile, os

    with tempfile.TemporaryDirectory() as tmpdir:
        cobol_file = os.path.join(tmpdir, "program.cob")
        exe_file = os.path.join(tmpdir, "program.exe" if os.name == "nt" else "program")

        with open(cobol_file, "w") as f:
            f.write(code)

        compile_cmd = ["cobc", "-x", cobol_file, "-o", exe_file]
        try:
            subprocess.run(compile_cmd, check=True, capture_output=True, text=True)
        except subprocess.CalledProcessError as e:
            return f"Compilation error:\n{e.stderr}"

        try:
            result = subprocess.run([exe_file], capture_output=True, text=True, check=True)
            return result.stdout
        except subprocess.CalledProcessError as e:
            return f"Runtime error:\n{e.stderr}"

In [98]:
def execute_cpp(code):
        import shutil

        write_output(code)
    
        compiler = shutil.which("clang++") or shutil.which("g++")
        if not compiler:
            return "Error: No C++ compiler (clang++ or g++) found in PATH."
    
        exe_name = "optimized.exe" if os.name == "nt" else "./optimized"
    
        compile_cmd = [compiler, "-Ofast", "-std=c++17", "-o", "optimized", "optimized.cpp"]
        run_cmd = [exe_name]
    
        try:
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [99]:
def execute_language(code, lang):
    if lang == "Python":
        return execute_python(code)
    elif lang == "Cobol":
        return execute_cobol(code)
    else:
        return "Unsupported language selected."

# UI

In [100]:
MODEL_OPTIONS = [
    MODEL_GPT_4O,
    MODEL_GPT_4_TURBO,
    MODEL_GPT_35_TURBO
]

with gr.Blocks(css=css) as ui:
    language = gr.Dropdown(["Python", "Cobol"], label="Select language", value="Python")
    gr.Markdown("## Convert code from Python to C++")
    
    with gr.Row():
        language_code = gr.Textbox(label="Language code:", lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    
    with gr.Row():
        model = gr.Dropdown(MODEL_OPTIONS, label="Select model", value=MODEL_GPT_4O)
    
    with gr.Row():
        convert = gr.Button("Convert code")
    
    with gr.Row():
        language_run = gr.Button("Run Selected Language")
        cpp_run = gr.Button("Run C++")
    
    with gr.Row():
        language_out = gr.TextArea(label="Language Output", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[language_code, model], outputs=[cpp])
    language_run.click(execute_language, inputs=[language_code, language], outputs=[language_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)